This expects you to have a spreadsheet with patients in rows, and demographic factors in columns
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Additional_Cov             | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

- If first time running this, uncomment and run the cell below

In [ ]:
# !pip install TableOne
# !pip install pandas

- Enter the paths to the excel/csv file containing your demographic data
- if using an excel file, provide the name of the sheet the specific data is in, otherwise set to none

- To use this, you need Python

In [ ]:
file_path = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/KiTH_Solutions/Research/Clinical Trial/study_metadata/all_performances.xlsx'
sheet = 'roca'

Run

In [ ]:
import pandas as pd
df = pd.read_excel(file_path, sheet)
df

Use the information printed below to select the columns you would like to put in a demographics table

In [ ]:
df.columns
sorted_columns = sorted(df.columns)
# Print the sorted column names
print(sorted_columns)

Enter the columns of interest.
- For columns which have words in them, also add them to 'categorical_columns = []'

In [ ]:
columns = ['Cube Actual', 'Cube Predicted', 'Infinity Actual', 'Infinity Predicted', 'Clock Actual', 'Clock Predicted']      
categorical_columns = ['Cube Actual', 'Cube Predicted', 'Infinity Actual', 'Infinity Predicted', 'Clock Actual', 'Clock Predicted']

- If you need to split a dataframe, say by treatment groups or arms, name the column to separate patients by using:
    - groupby = ['grouping_column']
- If you have a variable which is probably not normally distributed, name it here:
    - nonnormal = ['ordinal_column']
- If you want a column to be presented with a different name in the output table, you can write it here
    - labels={'column_to_rename': 'New Label For Table'}

In [ ]:
groupby = None
nonnormal = None
labels= None

Run it and evaluate the output. Do not proceed until it looks right. 

In [ ]:
from tableone import TableOne, load_dataset
mytable = TableOne(df, columns=columns, categorical=categorical_columns, groupby=groupby, nonnormal=nonnormal, rename=labels, pval=False if groupby is None else True)
print(mytable.tabulate(tablefmt = "fancy_grid"))

Where would you like to save to?

In [ ]:
out_dir = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Research/2023/roca/figures/drawing_table'

In [ ]:
import os
os.makedirs(out_dir, exist_ok=True)
mytable.to_excel(os.path.join(out_dir, 'table.xlsx'))
mytable.to_html(os.path.join(out_dir, 'table.html'))

Generate the Fancy Figure. Follow this installation

**macOS**
Install Homebrew (if not already installed):
Open Terminal and run:
- /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"
- brew install --cask mactex
- pdflatex --version


**Windows**
Download MiKTeX:
- Go to the MiKTeX download page and download the installer.

Run the Installer:
- Follow the installer instructions to complete the installation.

Verify Installation:
Open Command Prompt and check the pdflatex version:
- pdflatex --version

In [ ]:
import os
import subprocess
def run_pdflatex(tex_file):
    print('Converting latex to PDF using file:', tex_file)
    process = subprocess.run(['pdflatex', tex_file], 
                             cwd = os.path.dirname(tex_file),
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.stdout, process.stderr
    return stdout, stderr

def run_pdfpng(tex_file):
    base_file, _ = os.path.splitext(tex_file)
    print('Converting PDF to PNG using file:', tex_file)
    process = subprocess.run(["convert", "-density", "300", 
                              f'{base_file}.pdf', 
                              f'{base_file}.png'], 
                              stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.stdout, process.stderr
    return stdout, stderr

def run_latex(df, tex_file):
    df.to_latex(tex_file)
    with open(f'{tex_file}', "r") as f:
        data = f.read()
    data = (r"\documentclass{standalone}" + '\n' +
            r"\usepackage{booktabs}" + '\n' +
            r"\begin{document}" + '\n' +
            data +
            r"\end{document}")
    with open(f'{tex_file}', "w") as f:
        f.write(data)

def generate_latex_table(table, out_dir, manual=True):
    output_name = os.path.join(out_dir, 'table.tex')
    print("Generating file: ", output_name)
    os.makedirs(out_dir, exist_ok=True)
    if manual:
        run_latex(table, output_name)
    else:
        table.to_latex(output_name)
    
    _,_ = run_pdflatex(output_name)
    _,_ = run_pdfpng(output_name)
    

generate_latex_table(mytable, out_dir)

Enjoy
- Calvin 